### 店舗名変換テーブルの作成

In [60]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# カーソルを取得する
cursor = conn.cursor()

#自動コミットモードにする
conn.autocommit = True
conn.autocommit

True

In [61]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度
               FROM tenpo_master
               WHERE 経度 is NULL ''')
cols = [col.name for col in cursor.description]
non_latitune_df = pd.DataFrame(cursor.fetchall(),columns=cols)
non_latitune_df

,店舗名／法人名,都道府県,緯度,経度
0,アンダーツリー株式会社,大阪府,None,None
1,コシード株式会社,,None,None
2,株式会社千歳観光,,None,None
3,BASE株式会社,,None,None
4,ガイドワークス,,None,None
...,...,...,...,...
114,トキワホール,,None,None
115,パーラーデルダス,,None,None
116,Super D’station前橋大利根店,群馬県,None,None
117,yout小牧店,愛知県,None,None


In [22]:
non_latitune_df

,店舗名／法人名,都道府県,緯度,経度
0,アンダーツリー株式会社,大阪府,None,None
1,コシード株式会社,,None,None
2,株式会社千歳観光,,None,None
3,BASE株式会社,,None,None
4,ガイドワークス,,None,None
...,...,...,...,...
114,トキワホール,,None,None
115,パーラーデルダス,,None,None
116,Super D’station前橋大利根店,群馬県,None,None
117,yout小牧店,愛知県,None,None


In [62]:
cursor.execute('''SELECT dmm_name,hallnavi_name, prefecture, latitude
               FROM maptable
               WHERE prefecture = '神奈川県' 
               OR prefecture = '東京都'
                OR prefecture = '埼玉県'
                OR prefecture = '千葉県'
               ORDER BY prefecture''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
maptable_df['table_name'] = 'maptable'
maptable_df

,dmm_name,hallnavi_name,prefecture,latitude,table_name
0,テンイチ∞新松戸店,テンイチ∞新松戸店,千葉県,35.827183,maptable
1,ダイエー木更津店,ダイエー木更津店,千葉県,35.404198,maptable
2,アイオン稲毛東口店,アイオン稲毛東口店,千葉県,35.638142,maptable
3,ＭＧＭ芝山店,MGM芝山店,千葉県,35.732296,maptable
4,ミスターパチンコ流山店,ミスターパチンコ流山店,千葉県,35.851421,maptable
...,...,...,...,...,...
1468,新ガーデン戸塚,新！ガーデン戸塚,神奈川県,35.418137,maptable
1469,ワルキューレ茅ヶ崎店,【閉店】ワルキューレ茅ヶ崎店,神奈川県,35.329426,maptable
1470,共栄会館,共栄会館,神奈川県,35.144409,maptable
1471,大魔人 高座渋谷店,大魔人高座渋谷店,神奈川県,35.444393,maptable


In [63]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度 , "住所(ホールデータ)"
               FROM tenpo_master
               WHERE 都道府県 = '神奈川県' 
               OR 都道府県 = '東京都'
                OR 都道府県 = '埼玉県'
                OR 都道府県 = '千葉県'
               ORDER BY 都道府県''')
cols = [col.name for col in cursor.description]
tenpo_master_df = pd.DataFrame(cursor.fetchall(),columns=cols)
tenpo_master_df['table_name'] = 'tenpo_master'
tenpo_master_df.rename(columns={'店舗名／法人名':'dmm_name','都道府県':'prefecture'},inplace=True)
tenpo_master_df

,dmm_name,prefecture,緯度,経度,住所(ホールデータ),table_name
0,エクス・アリーナ 松戸,千葉県,35.762413,139.93720,千葉県松戸市高塚新田326‐5,tenpo_master
1,プリンス,千葉県,34.973930,137.16154,愛知県岡崎市井田南町7-7,tenpo_master
2,イーグルアール富里店,千葉県,35.749073,140.30844,千葉県富里市七栄1001-10,tenpo_master
3,キクヤ千葉店,千葉県,35.587177,140.12975,千葉県千葉市中央区末広5-7-1,tenpo_master
4,ＳＫＩＰ 館山店,千葉県,35.009476,139.87060,千葉県館山市湊231,tenpo_master
...,...,...,...,...,...,...
1215,ＨＡＰＰＹ Ｇ・ＦＵＪＩＳＡＷＡ,神奈川県,35.340324,139.48819,神奈川県藤沢市藤沢556,tenpo_master
1216,パラッツォ藤沢南店,神奈川県,35.338270,139.48947,神奈川県藤沢市南藤沢5-13,tenpo_master
1217,パールショップともえ平塚６０６,神奈川県,35.328053,139.34818,神奈川県平塚市紅谷町8-16,tenpo_master
1218,トワーズ藤沢店,神奈川県,35.403885,139.42984,神奈川県藤沢市葛原1806-6,tenpo_master


In [64]:
concat_df = pd.concat([tenpo_master_df,maptable_df],axis=0)
concat_df 

,dmm_name,prefecture,緯度,経度,住所(ホールデータ),table_name,hallnavi_name,latitude
0,エクス・アリーナ 松戸,千葉県,35.762413,139.93720,千葉県松戸市高塚新田326‐5,tenpo_master,NaN,NaN
1,プリンス,千葉県,34.973930,137.16154,愛知県岡崎市井田南町7-7,tenpo_master,NaN,NaN
2,イーグルアール富里店,千葉県,35.749073,140.30844,千葉県富里市七栄1001-10,tenpo_master,NaN,NaN
3,キクヤ千葉店,千葉県,35.587177,140.12975,千葉県千葉市中央区末広5-7-1,tenpo_master,NaN,NaN
4,ＳＫＩＰ 館山店,千葉県,35.009476,139.87060,千葉県館山市湊231,tenpo_master,NaN,NaN
...,...,...,...,...,...,...,...,...
1468,新ガーデン戸塚,神奈川県,NaN,NaN,NaN,maptable,新！ガーデン戸塚,35.418137
1469,ワルキューレ茅ヶ崎店,神奈川県,NaN,NaN,NaN,maptable,【閉店】ワルキューレ茅ヶ崎店,35.329426
1470,共栄会館,神奈川県,NaN,NaN,NaN,maptable,共栄会館,35.144409
1471,大魔人 高座渋谷店,神奈川県,NaN,NaN,NaN,maptable,大魔人高座渋谷店,35.444393


In [65]:
dupulicated_concat_df = concat_df[concat_df.duplicated(['dmm_name'],keep=False)].sort_values(['dmm_name'],ascending=False)
dupulicated_concat_df

,dmm_name,prefecture,緯度,経度,住所(ホールデータ),table_name,hallnavi_name,latitude
203,ＺＡＰ 蘇我店,千葉県,35.580640,140.13023,千葉県千葉市中央区今井2-10-10,tenpo_master,NaN,NaN
97,ＺＡＰ 蘇我店,千葉県,NaN,NaN,NaN,maptable,ZAP蘇我店,35.580513
1445,ＷＥＳＴ ＥＮＤ,神奈川県,NaN,NaN,NaN,maptable,WEST END あざみ野駅前,35.567982
963,ＷＥＳＴ ＥＮＤ,神奈川県,35.567978,139.55246,神奈川県横浜市青葉区あざみ野2-9-13,tenpo_master,NaN,NaN
161,Ｖｍａｘ津田沼,千葉県,NaN,NaN,NaN,maptable,Vmax(ブイマックス)津田沼,35.692940
...,...,...,...,...,...,...,...,...
1408,AQUA,神奈川県,NaN,NaN,NaN,maptable,グランドアクア,35.339668
913,AIS昭島店,東京都,35.712960,139.35994,,tenpo_master,NaN,NaN
1084,AIS昭島店,東京都,NaN,NaN,NaN,maptable,AIS昭島店,35.712959
1150,1ぱちZAP京急久里浜店,神奈川県,NaN,NaN,NaN,maptable,1ぱちZAP京急久里浜店,35.230984


In [66]:
a_list = list(tenpo_master_df['dmm_name'].unique())
b_list = list(dupulicated_concat_df[dupulicated_concat_df['table_name'] == 'maptable']['dmm_name'].unique())
serch_list = list(set(a_list) - set(b_list))
serch_list

['国際センター七光台店',
 'スロットムサシ',
 'マックス',
 'プリンス',
 'ワールド',
 'パチンコセブン',
 'パチンコパルコ',
 'ＪＵＭＢＯ',
 'ドリームタイガー',
 'ジャック＆ベティ',
 'シャレード',
 'パーラー吉兆',
 'アップル',
 '株式会社トリニティ',
 '人生劇場',
 'ムサシホール',
 '銀星',
 '7ホール',
 'ＭＡＸ',
 '京楽産業.株式会社',
 '株式会社西陣',
 '株式会社京楽産業ホールディングス',
 'ＡＢＣホール',
 'コスモ']

In [67]:
len(serch_list)

24

In [32]:

import re



pattern = '''(...??[都道府県])((?:旭川|伊達|石狩|盛岡|奥州|田村|南相馬|那須塩原|東村山|武蔵村山|羽村|十日町|上越|
富山|野々市|大町|蒲郡|四日市|姫路|大和郡山|廿日市|下松|岩国|田川|大村|宮古|富良野|別府|佐伯|黒部|小諸|塩尻|玉野|
周南)市|(?:余市|高市|[^市]{2,3}?)郡(?:玉村|大町|.{1,5}?)[町村]|(?:.{1,4}市)?[^町]{1,4}?区|.{1,7}?[市町村])(.+)'''


  



長野県
北安曇郡松川村
字東川原5723-22


In [ ]:
serch_list[1:]

In [59]:
import pandas as pd
import requests
import time
import re
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def insert_data_bulk(df,conn):
    insert_sql = f"""INSERT INTO maptable (parlar_map_id ,prefecture ,kintone_id ,shop_id ,hallnavi_name ,dmm_name ,anaslo_name ,latitude ,longitude ,adress ,created_at) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = conn.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    conn.commit()
    print("Insert bulk data")
    
hallnavi_scraping_error_tenpo_name_dict = {}
res = requests.get('https://chromedriver.storage.googleapis.com/LATEST_RELEASE')
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager(res.text).install(),options=options)
serched_df_3 = pd.DataFrame(index=[], columns=[])
hallnavi_tenpo_name_dict = {}
serched_df = pd.DataFrame(columns=['dmm_tenpo_name', 'prefecture','hallnavi_tenpo_name', 'url', 'longtitude','latitude'],index=[]) 

i = 1
for tenpo_name in serch_list[1:]:
    try:
        i += 1
        url = f"https://www.google.com/search?q=ホールナビ  {tenpo_name} "
        #print(i,tenpo_name)
        #print('データ取得中...30秒ほど時間がかかります。')
        browser.get(url)
        html = browser.page_source.encode('utf-8')
        soup = BeautifulSoup(html,'html.parser')
        class_group = soup.find_all(class_='yuRUbf')
        #print(len(class_group))
        # リンクとタイトル ,class_group is Noneを抽出しリストに追加するforループ
        for elem in class_group:
            # リンク
            url = elem.a.attrs.get('href')
            # タイトル
            title = elem.a.h3.text.split('｜')[0]
            print(title, url)
            break
        scraping_hallnavi_tenpo_name = title
        browser.get(url)
        
        html = browser.page_source.encode('utf-8')
        url_soup = BeautifulSoup(html,'html.parser')
        try:
            hallnavi_tenpo_name = url_soup.find_all(class_ = 'box_hole_view_hole_name')[0].text
        except Exception as e:
            hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
            print('エラー店舗',tenpo_name,e)
            continue
            
        dfs = pd.read_html(url)
        for df in dfs:
            try:
                check_text = df[0][0]
            except:
                continue
            if check_text == '店舗HP':
                display(df)
                break
        try:
            tenpo_adress = df[df[0] == '住所'][1].values[0]
        except Exception as e:
            hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
            print('エラー店舗',tenpo_name,e)
            continue
        #print(tenpo_adress)
        result = re.match(pattern, tenpo_adress)
        if result: #正規表現パターンにマッチした場合
            print(result.group(1)) #都道府県
            prefecture = result.group(1)
        else:
            prefecture = ''
        makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
        s_quote = urllib.parse.quote(tenpo_adress)
        response = requests.get(makeUrl + s_quote)
        #print(response.json()[0]["geometry"]["coordinates"])# [経度,緯度]
        try:
            longtitude = response.json()[0]["geometry"]["coordinates"][0]
            latitude = response.json()[0]["geometry"]["coordinates"][1]
        except Exception as e:
            hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
            print('エラー店舗',tenpo_name,e)
            continue
        #                  columns=['dmm_tenpo_name', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude']
        record_df  = pd.DataFrame([[tenpo_name,prefecture,hallnavi_tenpo_name,   url   , longtitude    ,latitude]],columns=['dmm_tenpo_name','prefecture', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude'])
        print(record_df)
        serched_df = pd.concat([serched_df,record_df],axis=0,ignore_index=True)
        
        parlar_map_id = 10000 + i
        prefecture = prefecture
        kintone_id = None
        shop_id = None
        hallnavi_name = hallnavi_tenpo_name
        dmm_name = tenpo_name
        anaslo_name = None
        latitude = latitude
        longitude = longtitude
        adress = tenpo_adress
        created_at = datetime.datetime.now()
        record_df = pd.DataFrame([[parlar_map_id,\
            prefecture,\
            kintone_id,
            shop_id,
            hallnavi_name,\
            dmm_name,\
            anaslo_name,\
            latitude,\
            longitude,\
            adress,\
            created_at]],columns=maptable_df.columns)
        record_df
        insert_data_bulk(record_df,conn)
        time.sleep(1)
        #break
    except Exception as e:
        hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
        print('エラー店舗',tenpo_name,e)
        #break
        continue
    # if i > 10:
    #     break
#data_df.to_csv(csv_file_name)

C:\Users\tsc95\AppData\Local\Temp\ipykernel_17176\2801476561.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager(res.text).install(),options=options)


レイトギャップ平和島店 https://hall-navi.com/hole_view?hid=143000600000001110


,0,1
0,店舗HP,DMMぱちタウン / P-WORLD
1,最寄駅,平和島
2,住所,東京都大田区平和島1-1-1
3,電話,03-5753-5777
4,パチンコ台数,586
5,パチンコ料金,[500円/125玉][100円/88玉]
6,スロット台数,406
7,スロット料金,[1000円/47枚][500円/88枚]
8,入場方法,抽選順。9:00抽選参加券配布
9,台移動,可能


東京都
  dmm_tenpo_name prefecture hallnavi_tenpo_name   
0        レイトギャップ        東京都         レイトギャップ平和島店  \

                                                 url  longtitude   latitude  
0  https://hall-navi.com/hole_view?hid=1430006000...  139.740753  35.584396  
Insert bulk data
夢屋五香店 https://hall-navi.com/hole_view?hid=270226100000052310


,0,1
0,店舗HP,DMMぱちタウン / P-WORLD
1,住所,千葉県松戸市常盤平5-23-1
2,電話,047-388-3600
3,パチンコ台数,266
4,パチンコ料金,[4][1]
5,スロット台数,189
6,スロット料金,[1000円/90枚][500円/90枚]
7,Line,店舗 LINE
8,その他,設置機種・営業状況はPWORLD等をご確認下さい。店休日には対応していません 店舗名でGoo...


...

神奈川県
  dmm_tenpo_name prefecture hallnavi_tenpo_name   
0          ピカデリー       神奈川県               ピカデリー  \

                                                 url  longtitude   latitude  
0  https://hall-navi.com/hole_view?hid=2400001000...  139.594833  35.461742  
Insert bulk data
ニュー後楽園JR千葉西口店 https://hall-navi.com/hole_view?hid=260003100000001410


,0,1
0,店舗HP,DMMぱちタウン / P-WORLD
1,住所,千葉県千葉市中央区新千葉1-4-1
2,電話,043-302-1616
3,パチンコ台数,332
4,パチンコ料金,[4][1.25]
5,スロット台数,170
6,スロット料金,[1000円/46枚]
7,その他,設置機種・営業状況はPWORLD等をご確認下さい。店休日には対応していません 店舗名でGoo...


千葉県
  dmm_tenpo_name prefecture hallnavi_tenpo_name   
0  ニュー後楽園JR千葉西口店        千葉県       ニュー後楽園JR千葉西口店  \

                                                 url  longtitude   latitude  
0  https://hall-navi.com/hole_view?hid=2600031000...  140.112839  35.612934  
Insert bulk data
新！ガーデン戸塚 https://hall-navi.com/hole_view?hid=245005300000028410


,0,1
0,店舗HP,DMMぱちタウン / P-WORLD
1,住所,神奈川県横浜市戸塚区上矢部町284-1
2,電話,045-814-5101
3,パチンコ台数,530
4,パチンコ料金,[4][1]
5,スロット台数,470
6,スロット料金,[1000円/46枚][1000円/182枚][1000円/455枚]
7,Line,店舗 LINE
8,その他,設置機種・営業状況はPWORLD等をご確認下さい。店休日には対応していません 店舗名でGoo...


神奈川県
  dmm_tenpo_name prefecture hallnavi_tenpo_name   
0        新ガーデン戸塚       神奈川県            新！ガーデン戸塚  \

                                                 url  longtitude   latitude  
0  https://hall-navi.com/hole_view?hid=2450053000...  139.534988  35.418137  
Insert bulk data


In [45]:
cursor.execute('''SELECT *
               FROM maptable
               WHERE prefecture = '神奈川県' 
               OR prefecture = '東京都'
                OR prefecture = '埼玉県'
                OR prefecture = '千葉県'
               ORDER BY prefecture''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)

maptable_df[:1]

,parlar_map_id,prefecture,kintone_id,shop_id,hallnavi_name,dmm_name,anaslo_name,latitude,longitude,adress,created_at
0,1369,千葉県,P04305,4305,ダイナム八街店,ダイナム八街店,ダイナム八街店,35.669998,140.339005,千葉県八街市八街に287-2,2023-03-18 05:37:47


In [44]:
import datetime
datetime.datetime.now()

datetime.datetime(2023, 8, 9, 4, 15, 8, 561380)

In [46]:
list(maptable_df.columns)

['parlar_map_id',
 'prefecture',
 'kintone_id',
 'shop_id',
 'hallnavi_name',
 'dmm_name',
 'anaslo_name',
 'latitude',
 'longitude',
 'adress',
 'created_at']

,parlar_map_id,prefecture,kintone_id,shop_id,hallnavi_name,dmm_name,anaslo_name,latitude,longitude,adress,created_at
0,10001,千葉県,None,None,【閉店】国際センター七光台店,国際センター七光台店,None,35.970905,139.859634,千葉県野田市吉春864,2023-08-09 04:21:41.949977


Insert bulk data


In [52]:
record_df

,parlar_map_id,prefecture,kintone_id,shop_id,hallnavi_name,dmm_name,anaslo_name,latitude,longitude,adress,created_at
0,10001,千葉県,None,None,【閉店】国際センター七光台店,国際センター七光台店,None,35.970905,139.859634,千葉県野田市吉春864,2023-08-09 04:21:41.949977


In [19]:
for i,row in serched_df.iterrows():
    print(row['longtitude'])
    longtitude = row['longtitude']
    latitude = row['latitude']
    dmm_tenpo_name = row['dmm_tenpo_name'].replace("'", "''")
    try:
        cursor.execute(f"""UPDATE tenpo_master
                    SET 経度 = {longtitude}, 緯度 = {latitude}
                WHERE  店舗名／法人名 = '{dmm_tenpo_name}' """)
        conn.commit()
    except Exception as e: 
        print("error", dmm_tenpo_name)
        break

141.114822
135.303955
139.336029
137.165039
138.242447
139.772293
139.890976
135.791534
135.477386
139.721359
130.767136
139.428146
134.541687
137.168671
137.24855
136.60643
136.243469
136.227402
137.956421
139.898041
137.044724
139.361755
139.624619
140.38266
131.545181
140.875778
131.594406
131.587341
139.810638
130.716385
139.702118
130.700653
130.479813
139.482193
138.889496
135.504074
138.297241
137.056519
138.814651
139.93605
140.184937
141.508011
140.774963
134.962418
141.389908
141.345749
138.238373
136.868637
139.741653
139.390549
137.154037
135.471359
137.159637
139.670425
135.454697
130.081085
139.05455
130.014816
140.20105
130.285843
135.495972
137.881088
136.609528
137.968048
130.736481
130.377777
140.943619
137.919174
135.165161
138.991531
135.516846
141.108749
130.480377
134.932495
140.876984
130.901108
130.498627
139.431335
135.712051
136.516022
137.063721
139.061996
130.95546
131.500198
137.035553
130.564194
130.565216
130.519501
136.9319
139.759338
135.940216
130.6463

In [168]:
df

,0,1
0,評価 -0.5,絆の森印西店


In [ ]:
try: 
    try:
        check_text = df[0][0]
    except:
        continue

In [153]:
dfs = pd.read_html(url)
for df in dfs:
    try:
        check_text = df[0][0]
    except:
        continue
    if check_text == '店舗HP':
        display(df)
        break

,0,1
0,店舗HP,DMMぱちタウン / P-WORLD
1,住所,千葉県千葉市中央区富士見2-4-5
2,電話,043-202-0077
3,パチンコ台数,667
4,パチンコ料金,[4][1]
5,スロット台数,370
6,スロット料金,[1000円/46枚][1000円/184枚]
7,Line,店舗 LINE
8,その他,設置機種・営業状況はPWORLD等をご確認下さい。店休日には対応していません 店舗名でGoo...


In [142]:
serched_df = pd.DataFrame(columns=['dmm_tenpo_name', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude'],index=[]) 
record_df  = pd.DataFrame([[tenpo_name,hallnavi_tenpo_name,   url   , longtitude    ,latitude]],columns=['dmm_tenpo_name', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude'])
serched_df = pd.concat([serched_df,record_df],axis=0,ignore_index=True)

In [144]:
serched_df

,dmm_tenpo_name,hallnavi_tenpo_name,url,longtitude,latitude
0,ＳＬＯＴ Ｇ８,SLOTG8,https://hall-navi.com/hole_view?hid=2991142000...,139.880066,35.343102


In [31]:


#リンク取得「a」タグ
for res in soup.find_all('a'):
    print(res.get('href'))
    break

/?sa=X&ved=0ahUKEwi_uNSPx7GAAxUZGjQIHc_2AN4QOwgC


In [16]:
import re
import requests

from bs4 import BeautifulSoup
googleSearch = 'https://www.google.co.jp/search'
response = requests.get(googleSearch, params={'q': f'ホールナビ  {tenpo_name} ','num':5})
html = response.text.encode()
bs4 = BeautifulSoup(html,'html.parser')
searchResults = bs4.select('.r > a')
for searchResult in searchResults:
    title = searchResult.text
    url = re.sub("\/url\?q=","",searchResult.get('href'))
    print('タイトル：',title)
    print('URL：',url)

In [18]:
searchResults

[]

In [ ]:

cursor.execute('''SELECT *
               FROM maptable
               WHERE prefecture = '神奈川県' AND latitude is NULL ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)





In [36]:
cursor = conn.cursor()
cursor.execute('''SELECT COUNT(DISTINCT 店舗名)
               FROM parlar_data
               WHERE 都道府県 = '神奈川県' or '埼玉県' ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
cursor.execute('''SELECT * 
               FROM schedule 
               ORDER BY 取得時間 DESC
               LIMIT 10''')
cols = [col.name for col in cursor.description]
schedule_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(schedule_df)